<a href="https://colab.research.google.com/github/nujudaly/QIS25/blob/main/CUDA_Q.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Intro to CUDA-Q NAVIDIA

In [1]:
!pip install cudaq


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.6/117.6 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 103.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 75.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 80.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.9/156.9 MB 6.3 MB/s eta 0:00:00
  Created wheel for cudaq: filename=cudaq-0.9.1-py3-none-any.whl size=6787 sha256=3e2729d9cb25dc135256cfb962716843909dc8e736d4f43c80d756577c8a5345
  Stored in directory: /root/.cache/pip/wheels/b0/ee/32/0ac9053509c0ba31e4bac6122454996a9a4e4d6523b1918d04
Successfully built cudaq
  Attempting uninstall: cupy-cuda12x


In [2]:
!pip install qutip


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.1/30.1 MB 57.6 MB/s eta 0:00:00


In [3]:
import cudaq
import qutip


In [4]:

# إنشاء الكيرنل
kernel = cudaq.make_kernel()
qubits = kernel.qalloc(2)

# تطبيق بوابات هادامارد و CNOT
kernel.h(qubits[0])
kernel.cx(qubits[0], qubits[1])

# قياس الكيوبتات
kernel.mz(qubits)

# تنفيذ الكيرنل وأخذ العينات
result = cudaq.sample(kernel)
print(result)


{ 00:513 11:487 }



In [5]:

# تحديد عدد الكيوبتات
qubit_count = 3

# تعريف الكيرنل
@cudaq.kernel
def ghz_kernel(qubit_count: int):
    # تخصيص الكيوبتات
    qvector = cudaq.qvector(qubit_count)

    # وضع أول كيوبت في حالة التراكب
    h(qvector[0])

    # تطبيق بوابة CNOT بين الكيوبتات
    for i in range(qubit_count - 1):
        x.ctrl(qvector[i], qvector[i + 1])

    # قياس الكيوبتات
    mz(qvector)


In [6]:
# تحديد عدد الكيوبتات
qubit_count = 3

# تشغيل الكيرنل وجمع النتائج
results = cudaq.sample(ghz_kernel, qubit_count)

# عرض النتائج
print(results)


{ 000:490 111:510 }



###Experminating with NAVIDIA **GPU**

In [7]:
pip install cudaq


In [8]:
pip install --upgrade cudaq


In [9]:
import cudaq
import timeit

# تعريف الكيرنل
@cudaq.kernel
def kernel():
    qubit_count = 25  # عدد الكيوبتات
    qubits = kernel.qalloc(qubit_count)  # تخصيص الكيوبتات

    # وضع أول كيوبت في حالة التراكب
    kernel.h(qubits[0])

    # تطبيق بوابات CNOT لإنشاء حالة GHZ
    for i in range(qubit_count - 1):
        kernel.cx(qubits[i], qubits[i + 1])

    # قياس الكيوبتات
    kernel.mz(qubits)

# إعداد الكود المراد تنفيذه
code_to_time = 'cudaq.sample(kernel, shots_count=1000000)'

# تنفيذ الكيرنل على المعالج المركزي (CPU)
try:
    cudaq.set_target('qpp-cpu')  # تعيين الهدف إلى CPU
    print('CPU time:')
    cpu_time = timeit.timeit(stmt=code_to_time, globals=globals(), number=1)
    print(f'{cpu_time:.5f} seconds')
except Exception as e:
    print(f"Error running on CPU: {e}")

# التحقق من وجود GPU وتنفيذ الكيرنل عليه
try:
    if cudaq.num_available_gpus() > 0:  # التحقق من عدد وحدات GPU المتاحة
        cudaq.set_target('nvidia')  # تعيين الهدف إلى GPU
        print('GPU time:')
        gpu_time = timeit.timeit(stmt=code_to_time, globals=globals(), number=1)
        print(f'{gpu_time:.5f} seconds')
    else:
        print("No GPU available.")
except Exception as e:
    print(f"Error running on GPU: {e}")


CPU time:
Error running on CPU: <ipython-input-9-0e32737f6b93>:8: error: Invalid function call - 'kernel' is unknown.
	 (offending source -> kernel.qalloc)
GPU time:
Error running on GPU: <ipython-input-9-0e32737f6b93>:8: error: Invalid function call - 'kernel' is unknown.
	 (offending source -> kernel.qalloc)


In [10]:
import torch

if torch.cuda.is_available():
    print("GPU is available!")
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
else:
    print("No GPU available.")


GPU is available!
GPU Name: Tesla T4


In [11]:
import cudaq

num_gpus = cudaq.num_available_gpus()
print(f"Number of GPUs available: {num_gpus}")
if num_gpus > 0:
    print("CUDA-Q GPU is available!")
else:
    print("No GPU available for CUDA-Q.")


Number of GPUs available: 1
CUDA-Q GPU is available!


In [12]:
import cudaq
print(cudaq.num_available_gpus())


1


In [13]:
cudaq.set_target('nvidia')


In [14]:
import cudaq

# إنشاء الكيرنل باستخدام make_kernel
kernel = cudaq.make_kernel()

# تخصيص كيوبت واحد
qubits = kernel.qalloc(1)

# تطبيق بوابة هادامارد على الكيوبت الأول
kernel.h(qubits[0])

# قياس الكيوبت
kernel.mz(qubits)

# تنفيذ الكيرنل
result = cudaq.sample(kernel, shots_count=1000)
print(result)


{ 0:490 1:510 }



In [15]:
print(cudaq.num_available_gpus())


1


In [16]:
import cudaq
import timeit

# إعداد الكيرنل
kernel = cudaq.make_kernel()
qubits = kernel.qalloc(25)  # تخصيص 25 كيوبت
kernel.h(qubits[0])  # وضع أول كيوبت في حالة التراكب
for i in range(24):  # تطبيق بوابات CNOT لإنشاء حالة GHZ
    kernel.cx(qubits[i], qubits[i + 1])
kernel.mz(qubits)  # قياس الكيوبتات

# إعداد الكود لاختبار الوقت
def execute_kernel():
    return cudaq.sample(kernel, shots_count=1_000_000)

# تشغيل الكيرنل على CPU
try:
    cudaq.set_target('qpp-cpu')  # تعيين الهدف إلى CPU
    print("Running on CPU...")
    cpu_time = timeit.timeit(execute_kernel, number=1)
    print(f"CPU time: {cpu_time:.2f} seconds")
except Exception as e:
    print(f"Error running on CPU: {e}")

# تشغيل الكيرنل على GPU إذا كان متوفراً
try:
    if cudaq.num_available_gpus() > 0:  # التحقق من وجود GPU
        cudaq.set_target('nvidia')  # تعيين الهدف إلى GPU
        print("Running on GPU...")
        gpu_time = timeit.timeit(execute_kernel, number=1)
        print(f"GPU time: {gpu_time:.2f} seconds")
    else:
        print("No GPU available.")
except Exception as e:
    print(f"Error running on GPU: {e}")


Running on CPU...
CPU time: 679.28 seconds
Running on GPU...
GPU time: 0.91 seconds


###Qiskit **GPU**

In [17]:
pip install qiskit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 78.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 kB 10.7 MB/s eta 0:00:00


In [18]:
pip install qiskit-ibm-runtime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 340.8/340.8 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.9/434.9 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.4/69.4 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 11.5 MB/s eta 0:00:00
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.27.2
    Uninstalling pydantic_core-2.27.2:
      Successfully uninstalled pydantic_core-2.27.2
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.10.5
    Uninstalling pydantic-2.10.5:
      Successfully uninstalled pydantic-2.10.5


In [19]:
pip install qiskit-aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 107.8 MB/s eta 0:00:00


In [20]:
pip install qiskit-aer-gpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.8/18.8 MB 19.4 MB/s eta 0:00:00


In [21]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
from qiskit_aer import AerSimulator
import time

# إنشاء دائرة GHZ مع 25 كيوبت
qc = QuantumCircuit(25)
qc.h(0)  # بوابة هادامارد على الكيوبت الأول
for i in range(24):  # تطبيق بوابات CNOT لإنشاء حالة GHZ
    qc.cx(i, i + 1)
qc.measure_all()  # قياس الكيوبتات

# إعداد المحاكي (Aer Simulator)
backend = AerSimulator()
# ترجمة الدائرة لتكون جاهزة للتنفيذ
transpiled_circuit = transpile(qc, backend)

# تشغيل الكيرنل على CPU أو GPU
try:
    print("Running Qiskit on Simulator...")
    start_time = time.time()
    job = backend.run(transpiled_circuit, shots=1_000_000)
    result = job.result()
    counts = result.get_counts()
    qiskit_time = time.time() - start_time
    print(f"Qiskit Results: {counts}")
    print(f"Qiskit Execution Time: {qiskit_time:.2f} seconds")
except Exception as e:
    print(f"Error running Qiskit: {e}")

# تحقق من دعم GPU
print(backend.configuration().to_dict())

Running Qiskit on Simulator...
Qiskit Results: {'1111111111111111111111111': 499635, '0000000000000000000000000': 500365}
Qiskit Execution Time: 25.45 seconds
{'backend_name': 'aer_simulator', 'backend_version': '0.15.1', 'n_qubits': 29, 'basis_gates': ['ccx', 'ccz', 'cp', 'crx', 'cry', 'crz', 'cswap', 'csx', 'cu', 'cu1', 'cu2', 'cu3', 'cx', 'cy', 'cz', 'diagonal', 'ecr', 'h', 'id', 'mcp', 'mcphase', 'mcr', 'mcrx', 'mcry', 'mcrz', 'mcswap', 'mcsx', 'mcu', 'mcu1', 'mcu2', 'mcu3', 'mcx', 'mcx_gray', 'mcy', 'mcz', 'multiplexer', 'p', 'pauli', 'r', 'roerror', 'rx', 'rxx', 'ry', 'ryy', 'rz', 'rzx', 'rzz', 's', 'sdg', 'store', 'swap', 'sx', 'sxdg', 't', 'tdg', 'u', 'u1', 'u2', 'u3', 'unitary', 'x', 'y', 'z', 'break_loop', 'continue_loop', 'delay', 'for_loop', 'if_else', 'initialize', 'kraus', 'qerror_loc', 'quantum_channel', 'reset', 'roerror', 'save_amplitudes', 'save_amplitudes_sq', 'save_clifford', 'save_density_matrix', 'save_expval', 'save_expval_var', 'save_matrix_product_state', 'save

###Pennylane **GPU**

In [22]:
pip install pennylane


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.1/56.1 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.0/930.0 kB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 71.5 MB/s eta 0:00:00


In [23]:
pip install pennylane-lightning


In [24]:
!pip install pennylane-lightning[gpu]


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.5 MB/s eta 0:00:00


In [25]:
!pip install --upgrade pip
!pip install pennylane pennylane-lightning[gpu] cuquantum-cu12

import pennylane as qml
from pennylane import numpy as np
import time

# إعداد الجهاز
n_qubits = 25  # عدد الكيوبتات
device = qml.device("lightning.gpu", wires=n_qubits, shots=1)  # لقطة واحدة لكل تشغيل

# تعريف الدائرة الكمومية
@qml.qnode(device)
def ghz_circuit():
    qml.Hadamard(wires=0)  # بوابة هادامارد على أول كيوبت
    for i in range(n_qubits - 1):  # بوابات CNOT
        qml.CNOT(wires=[i, i + 1])
    return qml.sample(wires=range(n_qubits))  # قياس جميع الكيوبتات

# تشغيل الدائرة وحساب الزمن
print("Running PennyLane...")
start_time = time.time()
shots = 1_000 # عدد اللقطات
counts = {"0" * n_qubits: 0, "1" * n_qubits: 0}  # إعداد العدادات

for _ in range(shots):
    result = ghz_circuit()  # تشغيل الدائرة
    result_str = "".join(map(str, result))  # تحويل النتيجة إلى سلسلة
    if result_str in counts:
        counts[result_str] += 1

execution_time = time.time() - start_time

# طباعة النتائج
print(f"PennyLane Results: {counts}")
print(f"PennyLane Execution Time: {execution_time:.2f} seconds")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 30.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
Running PennyLane...
PennyLane Results: {'0000000000000000000000000': 508, '1111111111111111111111111': 492}
PennyLane Execution Time: 72.84 seconds


#Technical Results and Analysis

1. NVIDIA CUDA-Q

    Execution Time:
        GPU: 0.91 seconds for 1,000,000 shots.
        CPU: 690 seconds for 1,000,000 shots.
    Results:
        Near-equal distribution between "000...0" and "111...1".
    Analysis:
        The performance on GPU NVIDIA was extremely fast due to efficient parallel computing.
        CUDA-Q is highly optimized for handling large numbers of shots.

2. Qiskit (Aer Simulator)

    Execution Time:
        CPU/GPU (Aer Simulator): 25.45 seconds for 1,000,000 shots.
    Results:
        Near-equal distribution between "000...0" and "111...1".
    Analysis:
        Qiskit was slower than CUDA-Q on GPU, but faster than CUDA-Q on CPU.
        Aer Simulator leverages software optimizations but doesn’t utilize GPU as effectively as CUDA-Q.

3. PennyLane

    Execution Time:
        lightning.gpu (GPU): 72.84 seconds for 1000 shots.
        
    Results:
        Results were as follows:
         '000...0': 508
         '111...1': 492
         The results are balanced, as expected for a GHZ state, which alternates equally between these two states.
    Analysis:
       Pennylane (lightning.gpu) is designed for hybrid quantum-classical workflows and provides flexibility for various quantum hardware. However, it is not optimized purely for speed in quantum circuit execution.
       The slower performance could be due to:
       1- Overhead in preparing and managing the GPU resources.
       2- Lack of specific optimizations for high-throughput quantum circuit sampling.

In [28]:
from prettytable import PrettyTable

# Create the PrettyTable instance
table = PrettyTable()

# Define the columns
table.field_names = [
    "Library",
    "Device",
    "Execution Time (1M Shots)",
    "Result Distribution",
    "Accuracy",
    "Strengths",
    "Weaknesses",
    "Use Case Suitability",
    "Remarks"
]

# Add rows with data
table.add_row([
    "Pennylane",
    "GPU (T4)",
    "72.84 seconds",
    "Balanced (508/492)",
    "High",
    "Flexible for hybrid workflows and quantum ML.",
    "Slower compared to CUDA-Q and Qiskit.",
    "Quantum ML and prototyping.",
    "Excellent for hybrid ML-quantum applications but slower."
])

table.add_row([
    "CUDA-Q",
    "GPU (T4)",
    "0.91 seconds",
    "Balanced",
    "High",
    "Fastest execution for large-scale circuits.",
    "Requires NVIDIA GPU for full optimization.",
    "High-performance quantum circuit execution.",
    "Best for fast and scalable simulations."
])

table.add_row([
    "CUDA-Q",
    "CPU",
    "690 seconds",
    "Balanced",
    "High",
    "Handles large circuits but slow.",
    "Extremely slow on CPU.",
    "Fallback option when no GPU is available.",
    "Not suitable for high-performance tasks."
])

table.add_row([
    "Qiskit Aer",
    "Aer Simulator (CPU/GPU)",
    "25.45 seconds",
    "Balanced (500,694/499,306)",
    "High",
    "Reliable simulation with extensive ecosystem.",
    "GPU utilization is limited compared to CUDA-Q.",
    "Research and general simulation needs.",
    "Good for prototyping but slower than CUDA-Q."
])

# Align columns
table.align = "l"

# Print the table
print(table)


+------------+-------------------------+---------------------------+----------------------------+----------+-----------------------------------------------+------------------------------------------------+---------------------------------------------+----------------------------------------------------------+
| Library    | Device                  | Execution Time (1M Shots) | Result Distribution        | Accuracy | Strengths                                     | Weaknesses                                     | Use Case Suitability                        | Remarks                                                  |
+------------+-------------------------+---------------------------+----------------------------+----------+-----------------------------------------------+------------------------------------------------+---------------------------------------------+----------------------------------------------------------+
| Pennylane  | GPU (T4)                | 72.84 seconds             